# Analyse de la performance d’une stratégie GARP (Growth at a Reasonable Price)

**Master 2 Finance Internationale**

Objectif :
Analyser la performance d’une stratégie de conviction GARP, en décomposant l’alpha,
et en évaluant la persistance du couple rendement–risque face à des benchmarks mondiaux.


In [5]:
# =========================
# INSTALLATION DES PACKAGES
# =========================

! pip install pandas numpy yfinance matplotlib seaborn statsmodels openpyxl scikit-learn



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


## 1. Préparation des données - Portefeuille GARP

Cette section vise à importer, nettoyer et harmoniser les données nécessaires à l’analyse empirique :
- Actions du portefeuille GARP (Yahoo Finance)
- Devise homogène : EUR
- Fréquence : mensuelle


In [8]:
# ======================================================
# 1. PREPARATION DES DONNEES – PORTEFEUILLE GARP
# ======================================================

# ----------------------
# 1.1 Librairies
# ----------------------
import pandas as pd
import numpy as np
import yfinance as yf
from pathlib import Path


# ----------------------
# 1.2 Univers d’investissement
# ----------------------
# Portefeuille GARP composé de 10 actions internationales
# La devise associée à chaque titre est précisée afin de
# permettre une conversion rigoureuse en euros

garp_stocks = {
    "ADYEN.AS": "EUR",
    "SLYG.DE": "EUR",
    "AMZN": "USD",
    "META": "USD",
    "GOOGL": "USD",
    "CSU.TO": "CAD",
    "MELI": "USD",
    "DNP.WA": "PLN",
    "NU": "USD",
    "KNSL": "USD"
}


# ----------------------
# 1.3 Période d’étude
# ----------------------
start_date = "2016-02-01"
end_date = "2026-01-31"


# ----------------------
# 1.4 Téléchargement des prix ajustés
# ----------------------
# Les prix ajustés (auto_adjust=True) incluent les dividendes
# et sont extraits depuis Yahoo Finance

prices = yf.download(
    tickers=list(garp_stocks.keys()),
    start=start_date,
    end=end_date,
    auto_adjust=True,
    progress=False
)["Close"]


# ----------------------
# 1.5 Passage en fréquence mensuelle
# ----------------------
# Les prix sont retenus en fin de mois afin d’être cohérents
# avec les benchmarks utilisés dans l’analyse ultérieure

prices_m = prices.resample("ME").last()


# ----------------------
# 1.6 Conversion des prix en euros
# ----------------------

# Tickers de change Yahoo Finance (devise locale -> EUR)
fx_tickers = {
    "USD": "USDEUR=X",
    "CAD": "CADEUR=X",
    "PLN": "PLNEUR=X"
}

# Téléchargement des taux de change
fx = yf.download(
    tickers=list(fx_tickers.values()),
    start=start_date,
    auto_adjust=True,
    progress=False
)["Close"]

# Passage en fréquence mensuelle
fx = fx.resample("ME").last()
fx.columns = fx_tickers.keys()

# Conversion des prix non libellés en EUR
prices_eur = prices_m.copy()

for ticker, currency in garp_stocks.items():
    if currency != "EUR":
        prices_eur[ticker] = prices_m[ticker] * fx[currency]


# ----------------------
# 1.7 Calcul des rendements logarithmiques
# ----------------------
# Les rendements logarithmiques sont privilégiés pour leur
# additivité temporelle et leur usage standard en finance empirique

returns_stocks = np.log(prices_eur / prices_eur.shift(1)).dropna()

# ----------------------
# 1.7 bis Traitement des valeurs manquantes
# ----------------------
# Les valeurs manquantes éventuelles sont remplacées par la
# médiane de chaque série afin de limiter l’impact des outliers
# et de préserver la structure de distribution des rendements

returns_stocks = returns_stocks.apply(
    lambda x: x.fillna(x.median())
)


# ----------------------
# 1.8 Construction du portefeuille GARP équipondéré
# ----------------------
# Chaque actif reçoit un poids constant identique

n_assets = returns_stocks.shape[1]
weights = np.repeat(1 / n_assets, n_assets)

# Rendement mensuel du portefeuille GARP
garp_portfolio_returns = returns_stocks.dot(weights)
garp_portfolio_returns.name = "GARP_Portfolio"


# ----------------------
# 1.9 Sauvegarde des données
# ----------------------
DATA_PROCESSED = Path("../data/processed")
DATA_PROCESSED.mkdir(exist_ok=True)

returns_stocks.to_csv(DATA_PROCESSED / "garp_stocks_returns.csv")
garp_portfolio_returns.to_csv(DATA_PROCESSED / "garp_portfolio_returns.csv")
